In [4]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import joblib
import imodelsx.process_results
import sys
from ridge_utils.features import qa_questions
sys.path.append('../experiments/')
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/single_question_may6'
experiment_filename = '../experiments/04_fit_single_question.py'

# load the results in to a pandas dataframe
r = imodelsx.process_results.get_results_df(results_dir)
cols_varied = imodelsx.process_results.get_experiment_keys(
    r, experiment_filename)
print('experiment varied these params:', cols_varied)
r = imodelsx.process_results.fill_missing_args_with_default(
    r, experiment_filename)

# get the breakdown of data in these groups
r.groupby(cols_varied).size()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 9/9 [00:12<00:00,  1.40s/it]

experiment varied these params: ['qa_embedding_model', 'qa_questions_version']


qa_embedding_model                   qa_questions_version
ensemble1                            v1                      1
                                     v2                      1
                                     v3_boostexamples        1
meta-llama/Meta-Llama-3-8B-Instruct  v1                      1
                                     v2                      1
                                     v3_boostexamples        1
mistralai/Mistral-7B-Instruct-v0.2   v1                      1
                                     v2                      1
                                     v3_boostexamples        1
dtype: int64

In [6]:
mets = [c for c in r.columns if 'corrs' in c and (
        'mean' in c or 'frac' in c)]

In [15]:
(
    r.groupby(cols_varied)[mets].mean()
    .style
    .background_gradient(
        cmap='viridis', axis=0)
    .format("{:.4f}")
)

In [ ]:
qs = qa_questions.get_questions(version='v1', full=True)

In [ ]:
r